In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install transformers
!pip install datasets

In [ ]:
import pandas as pd
import numpy as np

In [ ]:
# df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/Branch(일기)/Data/Crawling(brunch_일기_10291개).txt')
# df.head()

In [ ]:
df['내용'][10]

In [ ]:
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('\n', ' '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('.', '. '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace(',', ', '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('"', ''))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace("'", ''))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('?', '? '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('!', '! '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('  ', ' '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('  ', ' '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('  ', ' '))

In [ ]:
# import re
# df['내용'] = df['내용'].apply(lambda x : re.sub('[^ㄱ-ㅎㅏ-ㅣ가-힣A-Za-z0-9()., ]', '', x))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace(' .', ' '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('  ', ' '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('  ', ' '))
# df['내용'] = df['내용'].apply(lambda x : str(x).replace('  ', ' '))

In [ ]:
# df['내용'][30]

In [ ]:
# df['내용'].to_csv('/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/Branch(일기)/Data/Crawling(brunch_일기_10289개_전처리완료).txt', index = False)

In [ ]:
# pd.read_csv('/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/Branch(일기)/Data/Crawling(brunch_일기_10289개_전처리완료).txt')

In [ ]:
from transformers import TextDataset, DataCollatorForLanguageModeling
from transformers import GPT2LMHeadModel
from transformers import Trainer, TrainingArguments
from transformers import PreTrainedTokenizerFast

def load_dataset(file_path, tokenizer, block_size = 128):
    dataset = TextDataset(
        tokenizer = tokenizer,
        file_path = file_path,
        block_size = block_size,
    )
    return dataset

def load_data_collator(tokenizer, mlm = False):
    data_collator = DataCollatorForLanguageModeling(
        tokenizer = tokenizer,
        mlm = mlm,
    )
    return data_collator

def train(train_file_path, model_name, output_dir, overwrite_output_dir,
          per_device_train_batch_size, num_train_epochs, save_steps):
    tokenizer = PreTrainedTokenizerFast.from_pretrained(model_name,
                bos_token = '</s>', eos_token = '</s>', unk_token = '<unk>',
                pad_token = '<pad>', mask_token = '<mask>')
    train_dataset = load_dataset(train_file_path, tokenizer)
    data_collator = load_data_collator(tokenizer)

    tokenizer.save_pretrained(output_dir, legacy_format = False)
    model = GPT2LMHeadModel.from_pretrained(model_name)
    model.save_pretrained(output_dir)

    training_args = TrainingArguments(
        output_dir = output_dir,
        overwrite_output_dir = overwrite_output_dir,
        per_device_eval_batch_size = per_device_train_batch_size,
        num_train_epochs = num_train_epochs,
    )

    trainer = Trainer(
        model = model,
        args = training_args,
        data_collator = data_collator,
        train_dataset = train_dataset,
    )

    trainer.train()
    trainer.save_model()

train_file_path = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/Branch(일기)/Data/Crawling(brunch_일기_10289개_전처리완료).txt'
model_name = 'skt/kogpt2-base-v2'
output_dir = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/Branch(일기)/모델2(10289개 학습)'
overwrite_output_dir = False
per_device_train_batch_size = 8
num_train_epochs = 50.0
save_steps = 500

train(train_file_path = train_file_path,
      model_name = model_name,
      output_dir = output_dir,
      overwrite_output_dir = overwrite_output_dir,
      per_device_train_batch_size = per_device_train_batch_size,
      num_train_epochs = num_train_epochs,
      save_steps = save_steps
)

In [ ]:
from transformers import PreTrainedTokenizerFast, GPT2LMHeadModel
from tqdm.notebook import tqdm

def load_model(model_path):
  model = GPT2LMHeadModel.from_pretrained(model_path)
  return model

def load_tokenizer(tokenizer_path):
  tokenizer = PreTrainedTokenizerFast.from_pretrained(tokenizer_path)
  return tokenizer

def generate_text(sequence, max_lenth):
  model_path = '/content/drive/MyDrive/Colab Notebooks/팀프로젝트/빅데이터 지능형 서비스과정(최종프로젝트)/KoGPT2_FineTunning/Branch(일기)'
  model = load_model(model_path)
  tokenizer = load_tokenizer(model_path)
  ids = tokenizer.encode(f'{sequence},', return_tensors = 'pt')
  final_outputs = model.generate(
      ids,
      do_sample = True,
      max_length = max_length,
      pad_token_id = model.config.pad_token_id,
      top_k = 1,
      top_p = 0.90,
      repetition_penalty = 1.5,
  )
  return tokenizer.decode(final_outputs[0])
  # return tokenizer.decode(final_outputs[0], skip_special_tokens = True)

sequence = '즐거움 피자 맥주'
max_length = 256
sentence_list = []
# print('input : ' + sequence + ' ' + refer)
# for i in tqdm(range(5)):
sentence_list.append(generate_text(sequence, max_length))
sentence = generate_text(sequence, max_length)

In [ ]:
ch_sentence = sentence.split(sequence + ', ')[1:]
print(f'입력 값 : {sequence}')
# ch_sentence = ch_sentence[0].replace(',', ' ')
ch_sentence = ch_sentence[0].replace('\n', ' ')
ch_sentence = ch_sentence.replace('.', '. ')
ch_sentence = ch_sentence.replace('"', '')
ch_sentence = ch_sentence.replace('<unk>', '')
ch_sentence = ch_sentence.replace('?', '? ')
ch_sentence = ch_sentence.replace('!', '! ')
ch_sentence = ch_sentence.replace('  ', ' ')
ch_sentence = ch_sentence.replace('  ', ' ')
ch_sentence = ch_sentence.replace('  ', ' ')
ch_sentence